In [1]:
import numpy as np 
from Objects.sudoku import Sudoku
from Operators.fitness import fitness
from Operators.conflicts import box_conflicts, row_conflicts, col_conflicts
from Algorithms.search import hill_climbing, sim_annealing

import time
import tqdm

In [2]:
# Lets do grid search to find the best parameters for the hill climbing algorithm
hill_climbing_args={'num_neighbours': np.arange(1, 10, 1), 
                    'swap_number': np.arange(1, 10, 1)}

num_iterations = 20
num_combinations = 1

means = {}
stds = {}
unable_args = []

for combination in tqdm.tqdm(range(num_combinations)):
    unable = False
    # Get the random parameters
    # arg_dictionnary = {'num_neighbours': np.random.choice(hill_climbing_args['num_neighbours']),
    #                     'swap_number': np.random.choice(hill_climbing_args['swap_number']), 
    #                     'max_iterations': 100000,
    #                     'plateau_threshold': 100
    #                     }

    arg_dictionnary = {'num_neighbours': 10*combination+1,
                        'swap_number': 1, 
                        'max_iterations': 100000,
                        'plateau_threshold': 1000,
                        'verbose': 2
                        }
    
    temp_results = []
    for iteration in range(num_iterations):
        if unable:
            break
        start = time.time()
        try:
            Sudoku(hill_climbing_args=arg_dictionnary)
        except:
            unable = True
            continue

        finish = time.time()
        temp_results.append(finish-start)
    
    if unable:
        print('Unable to solve the puzzle with arguments: ', arg_dictionnary)
        unable_args.append(arg_dictionnary)
        continue

    means[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.mean(temp_results)
    stds[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.std(temp_results)

100%|██████████| 1/1 [00:00<?, ?it/s]

Unable to solve the puzzle with arguments:  {'num_neighbours': 1, 'swap_number': 1, 'max_iterations': 100000, 'plateau_threshold': 1000, 'verbose': 2}


In [2]:
sudoku = Sudoku(initial_board=np.zeros((9,9), dtype=int), fill_board='random')
sudoku.display()


 5  2  9  |  4  6  2  |  4  8  8 
 4  8  5  |  7  7  1  |  7  8  4 
 1  5  3  |  7  4  7  |  3  6  8 
----------|-----------|----------
 2  2  5  |  6  2  6  |  1  9  2 
 8  3  1  |  2  9  3  |  4  5  2 
 9  8  6  |  6  3  3  |  7  9  1 
----------|-----------|----------
 9  2  8  |  8  3  3  |  5  4  6 
 1  9  5  |  9  4  7  |  5  5  7 
 9  6  6  |  4  7  3  |  1  1  1 


In [ ]:
# Lets do grid search to find the best parameters for the hill climbing algorithm
hill_climbing_args={'num_neighbours': np.arange(1, 10, 1), 
                    'swap_number': np.arange(1, 10, 1)}

num_iterations = 20
num_combinations = 1

means = {}
stds = {}
unable_args = []

for combination in tqdm.tqdm(range(num_combinations)):
    unable = False
    # Get the random parameters
    # arg_dictionnary = {'num_neighbours': np.random.choice(hill_climbing_args['num_neighbours']),
    #                     'swap_number': np.random.choice(hill_climbing_args['swap_number']), 
    #                     'max_iterations': 100000,
    #                     'plateau_threshold': 100
    #                     }

    arg_dictionnary = {'num_neighbours': 10*combination+1,
                        'swap_number': 1, 
                        'max_iterations': 100000,
                        'plateau_threshold': 1000,
                        'verbose': 2
                        }
    
    temp_results = []
    for iteration in range(num_iterations):
        if unable:
            break
        start = time.time()
        try:
            Sudoku(hill_climbing_args=arg_dictionnary)
        except:
            unable = True
            continue

        finish = time.time()
        temp_results.append(finish-start)
    
    if unable:
        print('Unable to solve the puzzle with arguments: ', arg_dictionnary)
        unable_args.append(arg_dictionnary)
        continue

    means[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.mean(temp_results)
    stds[arg_dictionnary['num_neighbours'], arg_dictionnary['swap_number']] = np.std(temp_results)

100%|██████████| 1/1 [00:00<?, ?it/s]

Unable to solve the puzzle with arguments:  {'num_neighbours': 1, 'swap_number': 1, 'max_iterations': 100000, 'plateau_threshold': 1000, 'verbose': 2}


In [211]:
fitness_time = {}
num_combs = 100
num_iters = 20

for i in tqdm.tqdm(range(num_combs)):
    alpha = np.random.uniform(0.5, 0.99)
    c = np.random.randint(1, 50)
    L = np.random.randint(1, 50)

    temp_times = []
    temp_fitness = []

    for j in range(num_iters):
        start = time.time()
        fitness = sim_annealing(sudoku, verbose=0, alpha=alpha, c=c, L=L).fitness
        finish = time.time()
        temp_times.append(finish-start)
        temp_fitness.append(fitness)

    mean_time = np.mean(temp_times)
    mean_fitness = np.mean(temp_fitness)
    fitness_time[alpha, c, L, mean_fitness] = mean_time

100%|██████████| 100/100 [17:47<00:00, 10.68s/it]


In [224]:
# Create a dataframe from this 
import pandas as pd

df = pd.DataFrame.from_dict(fitness_time, orient='index', columns=['time'])
# Sepparate the index into columns
df['alpha'] = df.index.map(lambda x: x[0])
df['c'] = df.index.map(lambda x: x[1])
df['L'] = df.index.map(lambda x: x[2])
df['fitness'] = df.index.map(lambda x: x[3])
# Drop the index
df = df.reset_index().drop(columns='index')

In [228]:
df[df['fitness'] < 10]

,time,alpha,c,L,fitness
66,5.992550,0.980791,33,41,8.0
78,4.690322,0.985072,26,28,8.0
94,2.729274,0.978727,4,33,9.9


In [232]:
filled_sudoku = sim_annealing(sudoku, verbose=1, alpha=0.978727, c=4, L=33)
filled_sudoku = hill_climbing(filled_sudoku, verbose=1, num_neighbours=1, swap_number=1, max_iterations=100000, plateau_threshold=1000)

SA found with fitness 15
Hill climbing found [[2 5 4 8 1 9 6 7 3]
 [8 1 7 6 3 2 5 4 9]
 [9 3 6 7 4 5 2 8 1]
 [5 7 2 4 6 3 1 9 8]
 [1 9 3 2 8 7 4 6 5]
 [4 6 8 5 9 1 7 3 2]
 [3 8 5 1 7 6 9 2 4]
 [6 4 1 9 2 8 3 5 7]
 [7 2 9 3 5 4 8 1 6]]


In [198]:
filled_sudoku = hill_climbing(sudoku, verbose=1, num_neighbours=10, swap_number=1, max_iterations=100000, plateau_threshold=1000)

In [4]:
filled_sudoku.display()


 1  6  5  |  8  2  3  |  7  4  9 
 2  7  9  |  6  1  4  |  5  3  8 
 3  4  8  |  9  5  7  |  6  2  1 
----------|-----------|----------
 3  9  6  |  1  4  5  |  8  7  2 
 5  4  2  |  3  8  2  |  9  1  6 
 3  7  1  |  2  6  9  |  8  5  4 
----------|-----------|----------
 8  5  7  |  3  9  2  |  1  6  4 
 6  3  9  |  5  1  8  |  4  5  7 
 7  1  4  |  9  7  6  |  3  2  8 


In [5]:
# Iterate over all the positions to get where the conflict is 
for i in range(9):
    print(f'{i+1}th Box conflicts: {box_conflicts(filled_sudoku.board, i)}')
    print(f'{i+1}th Row conflicts: {row_conflicts(filled_sudoku.board, i)}')
    print(f'{i+1}th Col conflicts: {col_conflicts(filled_sudoku.board, i)}')

1th Box conflicts: 0
1th Row conflicts: 0
1th Col conflicts: 2
2th Box conflicts: 0
2th Row conflicts: 0
2th Col conflicts: 2
3th Box conflicts: 0
3th Row conflicts: 0
3th Col conflicts: 1
4th Box conflicts: 1
4th Row conflicts: 0
4th Col conflicts: 2
5th Box conflicts: 1
5th Row conflicts: 1
5th Col conflicts: 1
6th Box conflicts: 1
6th Row conflicts: 0
6th Col conflicts: 1
7th Box conflicts: 1
7th Row conflicts: 0
7th Col conflicts: 1
8th Box conflicts: 1
8th Row conflicts: 1
8th Col conflicts: 2
9th Box conflicts: 1
9th Row conflicts: 1
9th Col conflicts: 2


In [3]:
new_individual = Sudoku(hill_climbing_args={'max_iterations' : 100000, 
                                            'verbose': 3, 
                                            'num_neighbours': 100,
                                            'plateau_threshold': 10000})

Iteration 0 : Found a better solution with fitness: 74
Iteration 1 : Found a better solution with fitness: 71
Iteration 2 : Found a better solution with fitness: 67
Iteration 3 : Found a better solution with fitness: 63
Iteration 4 : Found a better solution with fitness: 60
Iteration 5 : Found a better solution with fitness: 57
Iteration 6 : Found a better solution with fitness: 54
Iteration 7 : Found a better solution with fitness: 52
Iteration 8 : Found a better solution with fitness: 50
Iteration 9 : Found a better solution with fitness: 48
Iteration 10 : Found a better solution with fitness: 46
Iteration 11 : Found a better solution with fitness: 44
Iteration 12 : Found a better solution with fitness: 43
Iteration 13 : Found a better solution with fitness: 42
Iteration 14 : Found a better solution with fitness: 39
Iteration 15 : Found a better solution with fitness: 37
Iteration 16 : Found a better solution with fitness: 36
Iteration 17 : Found a better solution with fitness: 35
It

AttributeError: 'Sudoku' object has no attribute 'initial_board'

In [5]:
new_individual

[[5 3 4 7 6 2 8 1 9]
 [2 1 8 9 3 5 7 6 4]
 [9 7 6 4 8 1 5 2 3]
 [4 5 1 8 7 3 6 9 2]
 [7 6 2 1 4 9 3 8 5]
 [8 9 3 5 2 6 4 7 1]
 [6 2 5 3 9 7 1 4 8]
 [1 8 9 6 5 4 2 3 7]
 [3 4 7 2 1 8 9 5 6]]

In [2]:
board = np.array([[9, 4, 7, 3, 2, 6, 5, 8, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 7, 3, 5, 9, 2, 1, 6, 8],
       [1, 2, 9, 8, 6, 4, 7, 3, 5],
       [5, 6, 8, 7, 1, 3, 4, 9, 2],
       [7, 9, 2, 4, 5, 8, 3, 1, 6],
       [6, 1, 5, 2, 3, 9, 8, 7, 4],
       [3, 8, 4, 6, 7, 1, 2, 5, 9]])
new_individual = Sudoku(initial_board=board)

In [19]:
climb = HillClimbing(new_individual.initial_board, new_individual.board)
climb.run(max_iterations=1000, swap_number=1, number_of_neighbours=1, verbose=True)

Iter 0 : Found a better board with fitness 25
Iter 3 : Found a better board with fitness 25
Iter 5 : Found a better board with fitness 23
Iter 6 : Found a better board with fitness 23
Iter 8 : Found a better board with fitness 21
Iter 10 : Found a better board with fitness 21
Iter 11 : Found a better board with fitness 21
Iter 12 : Found a better board with fitness 21
Iter 14 : Found a better board with fitness 18
Iter 15 : Found a better board with fitness 18
Iter 17 : Found a better board with fitness 18
Iter 18 : Found a better board with fitness 18
Iter 20 : Found a better board with fitness 18
Iter 21 : Found a better board with fitness 17
Iter 28 : Found a better board with fitness 17
Iter 30 : Found a better board with fitness 17
Iter 31 : Found a better board with fitness 15
Iter 36 : Found a better board with fitness 12
Iter 38 : Found a better board with fitness 12
Iter 39 : Found a better board with fitness 12
Iter 46 : Found a better board with fitness 12
Iter 47 : Found a 

In [11]:
climb.board

array([[9, 4, 7, 3, 2, 6, 5, 8, 1],
       [8, 3, 1, 9, 4, 5, 6, 2, 7],
       [2, 5, 6, 1, 8, 7, 9, 4, 3],
       [4, 7, 3, 5, 9, 2, 1, 6, 8],
       [1, 2, 9, 8, 6, 4, 7, 3, 5],
       [5, 6, 8, 7, 1, 3, 4, 9, 2],
       [7, 9, 2, 4, 5, 8, 3, 1, 6],
       [6, 1, 5, 2, 3, 9, 8, 7, 4],
       [3, 8, 4, 6, 7, 1, 2, 5, 9]])

In [13]:
new_individual.display()


 3  6  7  |  1  6  3  |  9  2  9 
 4  4  2  |  9  3  9  |  4  9  1 
 3  3  8  |  8  2  7  |  3  8  8 
----------|-----------|----------
 8  7  7  |  5  5  2  |  3  1  2 
 5  7  6  |  7  7  1  |  4  1  9 
 1  5  8  |  1  5  2  |  8  2  3 
----------|-----------|----------
 2  6  6  |  7  9  6  |  7  7  4 
 3  5  6  |  9  4  4  |  2  5  6 
 9  8  2  |  1  3  3  |  3  8  1 
